In [2]:
import json
import pandas as pd
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# df = pd.read_excel('../mims_drugs_work/mims_attribute_data.xlsx')
# df['Administration'].value_counts()

In [4]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 1.3 MB 38.1 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 65 kB 3.8 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=8510f95827a4707659bf0dd1aaa674062609945e932b567cb38b92189fdf9de2
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [5]:
import googletrans

print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [6]:
from googletrans import Translator
translator = Translator()
result = translator.translate('I m Rahul.',dest='hi')
print(result)

Translated(src=en, dest=hi, text=मैं राहुल हूँ।, pronunciation=main raahul hoon., extra_data="{'translat...")


In [7]:
import re

phrases=[]
engHindiDict = dict()

t1 = []
t2 = []
count=0
repeatcount = 0
separators1 = '; |, |and |\. |:|\(|\)'
separators2 = '\. '

In [8]:
import json
in_file = open('/content/drive/MyDrive/mims_drugs_work/englishHindi_json_out','r')
engHindiDict= json.load(in_file)
in_file.close()

In [9]:
def get_image(dat):
    image1=''
    imageURL = ''
    dname = [dat['name'].lower()]
    dnames = [i['name'].lower() for i in dat['synonyms']]
    # iname = [infobox['USAN'].lower()]
    # for i in dname + dnames +iname:
    for i in dname + dnames:
        row = chemstruct[chemstruct['names']==i]['images'].values
        if len(row)!=0:
            imageURL = row[0]
            image1=row[0].split('/')[-1]
            break
    return imageURL, image1

def get_schedule(dat):
    sch1=''
    dname = [dat['name'].lower()]
    dnames = [i['name'].lower() for i in dat['synonyms']]
    # iname = [infobox['USAN'].lower()]
    # for i in dname + dnames +iname:
    for i in dname + dnames:
        row = schedules[schedules['drugs']==i]['schedule'].values
        if len(row)!=0:
            sch1 = row[0]
            break
    return sch1


import pandas as pd

chemstruct = pd.read_csv('/content/drive/MyDrive/mims_drugs_work/publish/data/chemstruct.csv')
chemstruct['names']=chemstruct['names'].map(lambda x: x.lower())
schedules = pd.read_csv('/content/drive/MyDrive/mims_drugs_work/publish/data/schedules.csv')
schedules.sort_values(by=['drugs'],inplace=True)
schedules['drugs']=schedules['drugs'].map(lambda x: x.lower())


In [10]:
schedules

,drugs,schedule
7,"1,2,4 benzothiadiazine",G
152,2[cefdinir,H
32,5-phenylhydantoin,G
258,5-fluorouracil,H
63,abacavir,H
...,...,...
592,zolpidem,H
593,zopiclone,H
594,zuclopenthixol,H
265,decanoate,H


In [11]:
len(engHindiDict)

646266

In [12]:
def buildDictionary(attributeName,  separatorFlag = 1):
        # try:
          if attributeName not in data.keys():
            return
          t3a = data[attributeName]
          if t3a == None:
            return
          # print(t3a)
          # pre process/ cleaning operations
          if 'w\/' in t3a or '\.\[' in t3a:
            t3b = re.sub('w\/o', 'without', t3a)
            t3c = re.sub('w\/in', 'within', t3b)
            t3d = re.sub('w\/', 'with', t3c)
            t3 = re.sub('\.\[', '. [', t3d)
          else:
            t3 = t3a
          # print(t3)

          if separatorFlag == 1:
            separators = separators1
          else:
            separators = separators2

          t4 = re.split(separators,str(t3))
          for t5 in t4:
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text

        # except:
        #   pass

['absorption',
 'administration',
 'adverse reactions',

 'ao',
 'categories',
 'contraindications',
 'description',

 'dosages',
 'drug_interactions',
 'effects',
 'ep',
 'external_identifiers',
 'external_links',
 'fdalabel',
 'food_interactions',
 'halflife',
 'indication',
 'introduction',

 'direct-parent',
 'alternative-parent',
 'kingdom',
 'superclass',
 'subclass',

 'metabolism',
 'moa',
 'msds',
 'name',
 'overdosage',
 'patents',
 'pathway',
 'pharmacodynamics',
 'pregnancy category (us fda)',
 'proteinbinding',
 'reactions',
 'references',
 'roe',
 'salts',
 'special precautions',
 'state',
 'storage',
 'substituent',
 'synonyms',
 'synthesis_ref',
 'toxicity',
 'type',
 'vod',
 'class',
 'clearance'
 ]
 
NOT TO BE INCLUDED IN HINDI TRANSLATION
 [ 'primary-id',  'secondary-id', 'external_identifiers', 'external_links', 'unii',  'cas', 'fdalabel', 'msds' ]

In [ ]:
# build the dictionary of English to Hindi
linecount = 0

with open('/content/drive/MyDrive/mims_drugs_work/processed_data.jsonl','r') as f:
    for line in tqdm(f,total =14600):
        data = json.loads(line)

        # buildDictionary('name')
        # buildDictionary('description', 2)
        # buildDictionary('metabolism', 2)
        # buildDictionary('indication', 2)
        # buildDictionary('pharmacodynamics', 2)
        # buildDictionary('absorption',2)        
        # buildDictionary('Administration', 2)
        # buildDictionary('halflife')
        # buildDictionary('proteinbinding', 2)
        # buildDictionary('Special Precautions')
        # buildDictionary('Adverse Reactions')
        # buildDictionary('Contraindications')
        # "food_interactions": ["Avoid herbs and supplements with anticoagulant/antiplatelet activity. 
        # Examples include garlic, ginger, bilberry, danshen, piracetam, and ginkgo biloba."]
        # buildDictionary('food_interactions', 2)
        # buildDictionary('Overdosage', 2)
        # buildDictionary('patents')
        # buildDictionary('effects')
        # buildDictionary('introduction', 2)
        # buildDictionary('pathway')
        # buildDictionary('salts')
        # buildDictionary('vod')
        # buildDictionary('direct-parent')
        # buildDictionary('kingdom')
        # buildDictionary('superclass')
        # buildDictionary('class')
        # buildDictionary('subclass')
        # buildDictionary('state')
        # buildDictionary('moa', 2)
        # buildDictionary('toxicity', 2)
        # buildDictionary('ao', 2)
        # buildDictionary('roe', 2)
        # buildDictionary('synthesis_ref')
        # buildDictionary('drug_interactions', 2)


        
        # try:          
        #   t3 = data['alternative-parent']
        #   t4 = re.split(separators1,str(t3))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1
        # except:
        #   print("error in alternative-parent")


        # try:
        #   t3 = data['synonyms']
        #   for c2 in t3:
        #     t31 = c2['name']
        #     t4 = re.split(separators1,str(t31))
        #     for t5 in t4:
        #       if t5 not in engHindiDict.keys():
        #         result = translator.translate(t5, dest='hi')
        #         engHindiDict[t5] = result.text
        #         count = count +1
        #       else:
        #         repeatcount = repeatcount+1    
        # except:
        #   pass

          
          
        # # categories": [{"category": "Amino Acids, Peptides, and Proteins", "mesh-id": "D000602"}, 
        # t3 = data['categories']
        # for c2 in t3:
        #   t31 = c2['category']
        #   t4 = re.split(separators1,str(t31))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1

        # # "dosages": [{"form": "Injection, powder, for suspension", "route": "Intravenous", "strength": "50 mg"},
        # t3 = data['dosages']
        # for c2 in t3:
        #   t31 = c2['form']
        #   t4 = re.split(separators1,str(t31))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1              

        # "references": 
        # [{"ref-id": "A1", "pubmed-id": "16244762", "citation": "Smythe MA, Stephens JL, Koerber JM, Mattson JC: 
        # A comparison of lepirudin and argatroban outcomes. Clin Appl Thromb Hemost. 2005 Oct;11(4):371-4."}, 
        # {"ref-id": "A2", "pubmed-id": "16690967", "citation": "Tardy B, Lecompte T, Boelhen F, Tardy-Poncet B, Elalamy I, 
        #  Morange P, Gruel Y, Wolf M, Francois D, Racadot E, Camarasa P, Blouch MT, Nguyen F, Doubine S, Dutrillaux F, Alhenc-Gelas M, 
        #  Martin-Toutain I, Bauters A, Ffrench P, de Maistre E, Grunebaum L, Mouton C, Huisse MG, Gouault-Heilmann M, Lucke V: 
        #  Predictive factors for thrombosis and major bleeding in an observational study in 181 patients 
        #  with heparin-induced thrombocytopenia treated with lepirudin. Blood. 2006 Sep 1;108(5):1492-6. Epub 2006 May 11."}, 
        # t3 = data['references']
        # for c2 in t3:
        #   try:
        #     t31 = c2['citation']
        #     t4 = re.split(separators1,str(t31))
        #     for t5 in t4:
        #       if t5 not in engHindiDict.keys():
        #         result = translator.translate(t5, dest='hi')
        #         engHindiDict[t5] = result.text
        #         count = count +1
        #       else:
        #         repeatcount = repeatcount+1    
        #   except:
        #     try:
        #       t31 = c2['title']
        #       t4 = re.split(separators1,str(t31))
        #       for t5 in t4:
        #         if t5 not in engHindiDict.keys():
        #           result = translator.translate(t5, dest='hi')
        #           engHindiDict[t5] = result.text
        #           count = count +1
        #         else:
        #           repeatcount = repeatcount+1   
        #     except:
        #       print("Neither citation nor title found", c2)


        # drug_interactions": [{"drugbank-id": "DB06605", "name": "Apixaban", 
        # "description": "Apixaban may increase the anticoagulant activities of Lepirudin."}, 
        # {"drugbank-id": "DB06695", "name": "Dabigatran etexilate", 
        #  "description": "Dabigatran etexilate may increase the anticoagulant activities of Lepirudin."}, 
        #  {"drugbank-id": "DB01254", "name": "Dasatinib", 
        # "description": "The risk or severity of bleeding and hemorrhage can be increased when Dasatinib is combined with Lepirudin."}
        # t3 = data['drug_interactions']
        # for c2 in t3:
        #   t31 = c2['name']
        #   t5 = str(t31)
        #   if t5 not in engHindiDict.keys():
        #     result = translator.translate(t5, dest='hi')
        #     engHindiDict[t5] = result.text
        #     count = count +1
        #   else:
        #     repeatcount = repeatcount+1     

        #   t32 = c2['description']
        #   try:
        #     t33 = re.sub(str(t31), '$name', t32)
        #   except:
        #     t33 = t32
        #   t4 = re.split(separators1,str(t33))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1      

         #   t31 = c2['route']
        #   t4 = re.split(separators1,str(t31))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1   

        #   t31 = c2['strength']
        #   t4 = re.split(separators1,str(t31))
        #   for t5 in t4:
        #     if t5 not in engHindiDict.keys():
        #       result = translator.translate(t5, dest='hi')
        #       engHindiDict[t5] = result.text
        #       count = count +1
        #     else:
        #       repeatcount = repeatcount+1   



100%|█████████▉| 14594/14600 [37:23<00:00,  6.51it/s]


In [12]:
count, repeatcount

(0, 0)

In [32]:
#saves dictionary
import json
out_file = open('/content/drive/MyDrive/mims_drugs_work/englishHindi_json_out','w+')
json.dump(engHindiDict,out_file)
out_file.close()

In [13]:
def getAttributeDosage():
    try:
        alldosages = list()
        # "dosages": [{"form": "Injection, powder, for suspension", "route": "Intravenous", "strength": "50 mg"},
        t3 = data['dosages']
        for c2 in t3:
          dosagefinal = dict()
          finalHindiversion = ""
          t31 = c2['form']
          t4 = re.split(separators1,str(t31))
          for t5 in t4:
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text

          if finalHindiversion == "":
            finalHindiversion = engHindiDict[t5]
          else:
            finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
          dosagefinal['form'] = finalHindiversion

          finalHindiversion = ""  
          t31 = c2['route']
          t4 = re.split(separators1,str(t31))
          for t5 in t4:
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text
            if finalHindiversion == "":
              finalHindiversion = engHindiDict[t5]
            else:
              finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
          dosagefinal['route'] = finalHindiversion 

          finalHindiversion = ""  
          t31 = c2['strength']
          t4 = re.split(separators1,str(t31))
          for t5 in t4:
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text
            if finalHindiversion == "":
              finalHindiversion = engHindiDict[t5]
            else:
              finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
          dosagefinal['strength'] = finalHindiversion 
          alldosages.append(dosagefinal)
        drugFinal['dosages'] = alldosages        
    except:
      drugFinal['dosages'] = ''


In [14]:
def getAttributeName(attributeName, hindiName=None, separatorFlag = 1, compare = False):
    try:
        finalHindiversion = ""  
        t3 = data[attributeName]
        engName = data['name']

        if separatorFlag == 1:
          separators = separators1
        else:
          separators = separators2

        t4 = re.split(separators,str(t3))
        for t5 in t4:
          if t5 not in engHindiDict.keys():
            result = translator.translate(t5, dest='hi')
            engHindiDict[t5] = result.text

          # try embedding the name in a sentence to see if translation works
          if engHindiDict[t5] == t5 and t5 != '' and t5 != None and t5 == engName and ('[' not in t5):
            s0 = t5 + " is a drug"
            result = translator.translate(s0, dest='hi')
            s1 = result.text
            s2 = re.split(r' एक दवा है' , s1)
            engHindiDict[t5] = s2[0]
            # print(t5, s2[0])

          # try using the pronunciation in the translation
          if engHindiDict[t5] == t5 and t5 != '' and t5 != None and t5 == engName and ('[' not in t5):
            # instead of translating the name t5, we translate the pronunciation of previous translation
            t6 = result.pronunciation
            t7 = re.split(r' ek dava hai' , t6)
            result = translator.translate(t7[0], dest='hi')
            engHindiDict[t5] = result.text
            print(t5, engHindiDict[t5])

          if finalHindiversion == "":
            finalHindiversion = engHindiDict[t5]
          else:
            if engHindiDict != "":
              if separatorFlag == 1:
                finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
              else:
                finalHindiversion = finalHindiversion + engHindiDict[t5]
        try:
          finalHindiversion2 = finalHindiversion
          if (hindiName != None) and (hindiName != '') and engName in finalHindiversion2:
            finalHindiversion2 = re.sub(engName, hindiName, finalHindiversion)
        except:
          finalHindiversion2 = finalHindiversion

        if compare:
          n2 = getAttributeDirectTranslation(attributeName)
          print(finalHindiversion2, "\n DIRECT VERSION \n", n2)
        drugFinal[attributeName] = finalHindiversion2
        return finalHindiversion2
    except:
      drugFinal[attributeName] = ''

In [15]:
def loopUpReferences(refID, references):
  citeRef = ''
  for reff in references:
    if reff['ref-id'] == refID:
      if 'citation' in reff.keys():
        citeRef = '<ref>' + reff['citation'] + '</ref>'
      elif 'url' in reff.keys():
          citeRef = '<ref>' + reff['url'] + '</ref>'
      return citeRef

    if refID == 'MSDS':
      print("MSDS found")
      try:
        citeRef = '<ref>' + data['msds'] + '</ref>' 
      except:
        citeRef = ''
      return citeRef

    if refID == 'FDA label':
      print("FDA label found")
      try:
        citeRef = '<ref>' + data['fdalabel'] + '</ref>' 
      except:
        citeRef = ''
      return citeRef
  
  return citeRef

In [1]:
def getAttribute(attributeName, hindiName=None, separatorFlag = 1, compare = False):
    # try:
        finalHindiversion = ""  
        if attributeName not in data.keys():
          drugFinal[attributeName] = ''
          return
        t3a = data[attributeName]
        if t3a == None:
          drugFinal[attributeName] = ''
          return ''
        # print(t3a)
        # pre process/ cleaning operations
        if 'w\/' in t3a or '\.\[' in t3a:
          t3b = re.sub('w\/o', 'without', t3a)
          t3c = re.sub('w\/in', 'within', t3b)
          t3d = re.sub('w\/', 'with', t3c)
          t3 = re.sub('\.\[', '. [', t3d)
        else:
          t3 = t3a
        # print(t3)

        engName = data['name']

        if separatorFlag == 1:
          separators = separators1
        else:
          separators = separators2

        t4 = re.split(separators,str(t3))
        for t5 in t4:

          if t5 not in engHindiDict.keys():
            result = translator.translate(t5, dest='hi')
            engHindiDict[t5] = result.text


          if engHindiDict != "" and engHindiDict[t5] != "" and engHindiDict[t5] != None:
            if '[' in engHindiDict[t5]:
              # print(t5)
              # print(engHindiDict[t5])

              rfinal1 = engHindiDict[t5]
              # find references in text
              reff1 = re.findall(r'\[(.*?)\]', t5)  
              hindi_reff1 = re.findall(r'\[(.*?)\]', engHindiDict[t5])

              for i in range(len(reff1)):
                ref1 = reff1[i]
                if ref1 not in engHindiDict.keys():
                  result = translator.translate(ref1, dest='hi')
                  engHindiDict[ref1] = result.text                 
                hindi_ref1 = engHindiDict[ref1]

                # print(i, ref1, hindi_ref1)
                if ref1.isalpha() or ref1.isnumeric():
                  print(i, ref1 + " is not a reference \n")
                else:
                  if ref1.isalnum():
                    # print(ref1 + hindi_ref1+ " is a reference \n")
                    ref2 = loopUpReferences(ref1, data['references'])
                    # print(ref2, rfinal1)
                    if (ref2 != '' and ref2 != None):
                      replString = "["+hindi_ref1+"]"
                      # print("replace string -", replString)
                      if replString in rfinal1:
                        rfinal2 = rfinal1.replace(replString, ref2)
                        rfinal1 = rfinal2
                        # print(rfinal1)

                if hindi_ref1 == 'एमएसडीएस':
                  ref2 = loopUpReferences(ref1, data['references'])
                  print("replace with", ref2)
                  print("replace in", rfinal1)
                  if (ref2 != '' and ref2 != None):
                    replString = "["+hindi_ref1+"]"
                    rfinal2 = rfinal1.replace(replString, ref2)
                    rfinal1 = rfinal2

                if hindi_ref1 == "एफडीए लेबल":
                  ref2 = loopUpReferences(ref1, data['references'])
                  print("replace with", ref2)
                  print("replace in", rfinal1)
                  if (ref2 != '' and ref2 != None):
                    replString = "["+hindi_ref1+"]"
                    rfinal2 = rfinal1.replace(replString, ref2)
                    rfinal1 = rfinal2

              engHindiDict[t5] = rfinal1
              print(rfinal1)
              # print(finalHindiversion.strip()[-1])
          if finalHindiversion == "":
            finalHindiversion = engHindiDict[t5]
          else:
            if separatorFlag == 1:
              finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
            else:
              if finalHindiversion.strip()[-1] != '।':
                # print("sentence marker not found")
                finalHindiversion = finalHindiversion + " । " + engHindiDict[t5]
              else:
                # print("sentence marker FOUND")
                finalHindiversion = finalHindiversion + engHindiDict[t5]
        try:
          finalHindiversion2 = finalHindiversion
          if (hindiName != None) and (hindiName != '') and engName in finalHindiversion2:
            finalHindiversion2 = re.sub(engName, hindiName, finalHindiversion)
        except:
          finalHindiversion2 = finalHindiversion

        if compare:
          n2 = getAttributeDirectTranslation(attributeName)
          print(finalHindiversion2, "\n DIRECT VERSION \n", n2)

        drugFinal[attributeName] = finalHindiversion2

        # print(finalHindiversion2)
        return finalHindiversion2
    # except:
    #   drugFinal[attributeName] = ''

In [16]:
def getAttributeDirectTranslation(attributeName, hindiName=None):
    try:
        finalHindiversion = ""  
        t3 = data[attributeName]
        engName = data['name']
        finalHindiversion = translator.translate(t3, dest='hi')

        try:
          finalHindiversion2 = finalHindiversion
          if (hindiName != None) and (hindiName != '') and engName in finalHindiversion2:
            finalHindiversion2 = re.sub(engName, hindiName, finalHindiversion)
          # print(finalHindiversion, "\n", finalHindiversion2)

        except:
          finalHindiversion2 = finalHindiversion

        drugFinal[attributeName] = finalHindiversion2
        return finalHindiversion2
    except:
      drugFinal[attributeName] = ''

In [17]:
# attributes2 = ['drug_interactions', 'categories']

def getAttributeDrugInteractions():

    try:
        allDrugInteractions = list()
   
        # drug_interactions": [{"drugbank-id": "DB06605", "name": "Apixaban", 
        # "description": "Apixaban may increase the anticoagulant activities of Lepirudin."}, 
        # {"drugbank-id": "DB06695", "name": "Dabigatran etexilate", 
        #  "description": "Dabigatran etexilate may increase the anticoagulant activities of Lepirudin."}, 
        #  {"drugbank-id": "DB01254", "name": "Dasatinib", 
        # "description": "The risk or severity of bleeding and hemorrhage can be increased when Dasatinib is combined with Lepirudin."}
        t3 = data['drug_interactions']
        for c2 in t3:
          drugInterFinal = dict()
          drugInterFinal['drugbank-id'] = c2['drugbank-id']
          t31 = c2['name']
          finalHindiversion = "" 
          t5 = str(t31)
          if t5 not in engHindiDict.keys():
            result = translator.translate(t5, dest='hi')
            engHindiDict[t5] = result.text

          if finalHindiversion == "":
            finalHindiversion = engHindiDict[t5]
          else:
            finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
          drugInterFinal['name'] = finalHindiversion    

          t32 = c2['description']
          finalHindiversion = "" 
          try:
            t33 = re.sub(str(t31), '$name', t32)
          except:
            t33 = t32          
          t4 = re.split(separators1,str(t33))
          for t5 in t4:          
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text

            if finalHindiversion == "":
              finalHindiversion = engHindiDict[t5]
            else:
              finalHindiversion = finalHindiversion + "," + engHindiDict[t5]
          # print(finalHindiversion)    

          # we have to replace $name with hindi version of t33, TO BE DONE
          try:
            finalHindiversion2 = re.sub('\$name', drugInterFinal['name'], finalHindiversion)
          except:
            finalHindiversion2 = finalHindiversion  


          drugInterFinal['description'] = finalHindiversion2    
          allDrugInteractions.append(drugInterFinal)
        drugFinal['drug_interactions'] = allDrugInteractions
    except:
        drugFinal['drug_interactions'] = ''

def getAttributeCategories():
    try:
        allcategories = list()
        # categories": [{"category": "Amino Acids, Peptides, and Proteins", "mesh-id": "D000602"}, 
        t3 = data['categories']
        for c2 in t3:
          categoryFinal = dict()
          finalHindiversion = ""
          t31 = c2['category']
          t4 = re.split(separators1,str(t31))
          for t5 in t4:
            if t5 not in engHindiDict.keys():
              result = translator.translate(t5, dest='hi')
              engHindiDict[t5] = result.text

            if finalHindiversion == "":
              finalHindiversion = engHindiDict[t5]
            else:
              finalHindiversion = finalHindiversion + "," + engHindiDict[t5]

          categoryFinal['category'] = finalHindiversion 
          categoryFinal['mesh-id'] = c2['mesh-id']
          allcategories.append(categoryFinal)
        drugFinal['categories'] = allcategories
    except:
        drugFinal['categories'] = ''            

In [18]:
# construct the sentences in Hindi from the dictionary
count = 0
import json
in_file = open('/content/drive/MyDrive/mims_drugs_work/englishHindi_json_out','r')
engHindiDict= json.load(in_file)
in_file.close()

drugFinal = dict()
finalData = list()
sampleData = list()
with open('/content/drive/MyDrive/mims_drugs_work/processed_data.jsonl','r') as f:
    for line in tqdm(f,total =14600):
      data = json.loads(line)
      drugFinal = dict()
      engName = data['name']
      if (engName  == 'Pravastatin') or (engName == 'Bivalirudin'):
        imageURL = ''
        img1 = ''
        n1 = getAttributeName('name')
        print(n1)
        try:
          drugFinal['references'] = data['references']
        except:
          drugFinal['references'] = ''

        getAttribute('description', n1, 2)
        getAttribute('metabolism', n1, 2)
        getAttribute('indication', n1, 2)
        getAttribute('pharmacodynamics', n1, 2)
        getAttribute('absorption', n1, 2)        
        getAttributeDosage()
        getAttribute('Administration', n1)
        getAttribute('halflife', n1)
        getAttribute('proteinbinding', n1, 2)
        getAttribute('Special Precautions', n1, 1)
        getAttribute('Adverse Reactions', n1)
        getAttribute('Contraindications', n1)
        getAttribute('food_interactions', n1, 2)

        imageURL, img1 = get_image(data)
        if img1:
          drugFinal['imageURL'] = imageURL
          drugFinal['image'] = img1
        else:
          drugFinal['imageURL'] = ''
          drugFinal['image'] = ''

        sch1 = get_schedule(data)
        if sch1:
          drugFinal['schedule'] = sch1
        else:
          drugFinal['schedule'] = ''

        # getAttributeDrugInteractions()
        getAttributeCategories()
        getAttribute('Storage')
        getAttribute('Drug Interactions', n1, 2)
        getAttribute('Overdosage',n1, 2)
        getAttribute('patents')
        getAttribute('effects')
        getAttribute('introduction', n1, 2)
        getAttribute('pathway')
        getAttribute('salts')
        getAttribute('vod')
        getAttribute('direct-parent')
        getAttribute('kingdom')
        getAttribute('superclass')
        getAttribute('class')
        getAttribute('subclass')
        getAttribute('state')
        getAttribute('moa', n1, 2)
        getAttribute('toxicity', n1, 2)
        getAttribute('ao', n1, 2)
        getAttribute('roe', n1, 2)
        getAttribute('synthesis_ref')

#  [ 'primary-id',  'secondary-id', 'external_identifiers', 'external_links', 'unii',  'cas', 'fdalabel', 'msds' ]
        drugFinal['primary-id'] = data['primary-id']
        if 'secondary-id' in data.keys():
          drugFinal['secondary-id'] = data['secondary-id']
        else:
          drugFinal['secondary-id'] = ''
        drugFinal['external_links'] = data['external_links']
        drugFinal['external_identifiers'] = data['external_identifiers']
        drugFinal['unii'] = data['unii']
        drugFinal['cas'] = data['cas']

        if 'fdalabel' in data.keys():
          drugFinal['fdalabel'] = data['fdalabel']
        else:
          drugFinal['fdalabel'] = ''
        if 'msds' in data.keys():
          drugFinal['msds'] = data['msds']
        else:
          drugFinal['msds'] = ''

        drugFinal['engName'] = data['name']

        try:
          drugFinal['pregnancy category (us fda)'] = data['pregnancy category (us fda)']
        except:
          drugFinal['pregnancy category (us fda)'] = ''
        try:
          drugFinal['atc'] = data['atc']
        except:
          drugFinal['atc'] = ''
        try:
          drugFinal['synonyms'] = data['synonyms']
        except:
          drugFinal['synonyms'] = ''
        try:
          drugFinal['mmass'] = data['mmass']
        except:
          drugFinal['mmass'] = ''
        try:
          drugFinal['amass'] = data['amass']
        except:
          drugFinal['amass'] = ''
        try:
          drugFinal['ep'] = data['ep']
        except:
          drugFinal['ep'] = ''

        print(drugFinal['toxicity'])
        finalData.append(drugFinal)

        sampleData.append(drugFinal)


  0%|          | 1/14600 [00:00<2:01:19,  2.01it/s]

बिवलिरुदिन
औसत स्थिर अवस्था सांद्रता [12,3] +/- [1,7] एमसीजी/एमएल है जो 1 मिलीग्राम/किलोग्राम के अंतःशिरा बोल्ट के प्रशासन के बाद [2,5] मिलीग्राम/किग्रा/घंटा 4 से अधिक दिया जाता है। घंटे।
['* सामान्य गुर्दे समारोह
['क्रिएटिनिन क्लीयरेंस 10-29एमएल/मिनट
[3,5]ज']]
['तीव्र स्टेंट घनास्त्रता'
['इचिनेशिया से बचें।', 'जड़ी-बूटियों और एंटीकोआगुलेंट/एंटीप्लेटलेट गतिविधि वाले सप्लीमेंट्स से बचें।
[{'संख्या'
[{'एसएमपीडीबी-आईडी'
[0.2] एल/किग्रा
['मनुष्य और अन्य स्तनधारी']


  0%|          | 6/14600 [00:01<49:33,  4.91it/s]  

[2007,]
ग्लीसन एट अल द्वारा एक अध्ययन के आधार पर, 24 घंटे की अवधि में अंतःशिरा जलसेक के माध्यम से चूहों को प्रशासित बिवलिरुडिन के लिए नो-अवलोकन-प्रतिकूल-प्रभाव स्तर (एनओएईएल), 2000 मिलीग्राम / किग्रा / 24 घंटे था।
प्रवास्टैटिन
0 mevastatin is not a reference 

Pravastatin [mevastatin] का 6-अल्फा-हाइड्रॉक्सी एसिड रूप है।
2 mevastatin is not a reference 

लिमिटेड, हालांकि, पहला स्वीकृत प्रवास्टैटिन उत्पाद ब्रिस्टल मायर्स स्क्विब द्वारा विकसित किया गया था और एफडीए [1991,] <ref>https://www.accessdata.fda.gov/scripts/cder/daf/index.cfm?event=overview.process&ApplNo=019898</ref> में अनुमोदित था।
['प्रवास्टैटिन कोरोनरी हृदय रोग के नैदानिक ​​​​सबूत के बिना कोरोनरी घटनाओं की प्राथमिक रोकथाम के लिए संकेत दिया गया है हाइपरकोलेस्ट्रोलेमिक रोगियों
इस संकेत में कोरोनरी डेथ, मायोकार्डियल इंफार्क्शन, मायोकार्डियल रिवास्कुलराइजेशन प्रक्रियाओं, स्ट्रोक, और स्ट्रोक / क्षणिक इस्केमिक हमले के साथ-साथ कोरोनरी एथेरोस्क्लेरोसिस की प्रगति को धीमा करके कुल मृत्यु दर के जोखिम को कम करना शामिल है। <ref>Frishman W

  4%|▍         | 606/14600 [00:04<01:11, 194.63it/s]

प्रवास्टैटिन के वितरण की रिपोर्ट की गई स्थिर-अवस्था की मात्रा [0,5] एल/किलोग्राम होने की सूचना है। <ref>Singhvi SM, Pan HY, Morrison RA, Willard DA: Disposition of pravastatin sodium, a tissue-selective HMG-CoA reductase inhibitor, in healthy subjects. Br J Clin Pharmacol. 1990 Feb;29(2):239-43.</ref> बच्चों में यह फार्माकोकेनेटिक पैरामीटर 31-37 मिलीलीटर/किग्रा तक पाया गया था।
0 MSDS is not a reference 

MSDS found
replace with <ref>//s3-us-west-[2,]amazonaws.com/drugbank/msds/DB[00175,]pdf?1556745701</ref>
replace in चूहों में प्रवास्टैटिन की रिपोर्ट की गई मौखिक एलडी 50 8939 मिलीग्राम/किलोग्राम है। <ref>//s3-us-west-[2,]amazonaws.com/drugbank/msds/DB[00175,]pdf?1556745701</ref> कोई महत्वपूर्ण अतिदेय रिपोर्ट नहीं है, हालांकि, अधिक मात्रा के मामले में, प्रयोगशाला निगरानी और सहायक उपायों के साथ रोगसूचक उपचार की सिफारिश की जाती है। [एफडीए लेबल] कार्सिनोजेनिक अध्ययनों में, प्रवास्टैटिन के उच्च खुराक प्रशासन से पुरुषों में हेपेटोसेलुलर कार्सिनोमा और महिलाओं में फेफड़े के कार्सिनोमा की घटनाओ

100%|█████████▉| 14594/14600 [00:05<00:00, 2704.25it/s] 


In [19]:
import json
out_file = open('/content/drive/MyDrive/mims_drugs_work/publish/m1_sample_json_out','w+')
json.dump(sampleData,out_file)
out_file.close()

In [ ]:
import json
out_file = open('/content/drive/MyDrive/mims_drugs_work/publish/m2_json_out','w+')
json.dump(finalData,out_file)
out_file.close()